In [ ]:
import RAE
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from torch import nn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from itertools import product
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pickle
import matplotlib.pyplot as plt

import plotly.express as px
from pyproj import Transformer

import os
import joblib

import importlib
import json
import itertools

In [ ]:
def visualize_trajectories(df, color_by='MMSI', title=None, zoom=5, height=800):
    
    transformer = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(df['UTM_x'].values, df['UTM_y'].values)
    
    if 'Trajectory' in df.columns:
        vis_cols = ['Trajectory', 'MMSI', 'Timestamp', 'UTM_x', 'UTM_y']
    else:
        vis_cols = ['MMSI', 'Timestamp', 'UTM_x', 'UTM_y']

    if color_by not in vis_cols:
        vis_cols.append(color_by)

    if 'SOG' in df.columns and 'SOG' not in vis_cols:
        vis_cols.append('SOG')
    
    vis_df = df[vis_cols].copy()
    vis_df['Longitude'] = lon
    vis_df['Latitude'] = lat
    
    # Generate title if not provided
    if title is None:
        date_min = vis_df['Timestamp'].min().date()
        date_max = vis_df['Timestamp'].max().date()
        title = f"Ship Trajectories - {date_min} to {date_max}"
    
    # Create visualization with trajectories using lat/lon on a map
    fig = px.line_map(
        vis_df.sort_values('Timestamp'),
        lat="Latitude",
        lon="Longitude",
        color=color_by,
        hover_data=["MMSI", "Timestamp", "SOG"] if "SOG" in vis_df.columns else ["MMSI", "Timestamp"],
        zoom=zoom,
        title=title
    )
    
    fig.update_layout(
        mapbox_style="open-street-map",
        showlegend=False,  # Hide legend since there can be many trajectories
        height=height
    )
    
    print(f"✓ Visualization complete - colored by '{color_by}'")
    
    return fig

In [ ]:
train = pd.read_parquet('../../data/aisdk/train/aisdk_2025')
val = pd.read_parquet('../../data/aisdk/val/aisdk_2025')

In [ ]:
obj = joblib.load(os.path.join("..", "..", "data", "aisdk", "scaler", "scaler_aisdk_2025.pkl"))

saved_feature_names = None

if isinstance(obj, StandardScaler):
    scaler = obj
    # If you stored feature names separately, keep as None here
    print("Loaded StandardScaler object from joblib.")
elif isinstance(obj, dict):
    scaler = StandardScaler()
    scaler.mean_ = np.asarray(obj['mean'])
    scaler.scale_ = np.asarray(obj['scale'])
    saved_feature_names = obj.get('feature_names', None)
    print("Reconstructed StandardScaler from dict params.")
else:
    raise TypeError(f"Unexpected scaler payload type: {type(obj)}")

print("Scaler ready. n_features:", scaler.mean_.shape[0],
      "| feature names:", (list(saved_feature_names) if saved_feature_names is not None else "None"))

In [ ]:
# Un-scale train and val DataFrames for visualization
def inverse_scale_df(df, scaler, cols, saved_feature_names=None):
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return df.copy(), []
    Xs = df[cols].to_numpy()
    mean = np.asarray(scaler.mean_)
    scale = np.asarray(scaler.scale_)
    # Align mean/scale to column order if saved feature names exist
    if saved_feature_names is not None:
        name_to_idx = {name: i for i, name in enumerate(saved_feature_names)}
        idx = []
        for c in cols:
            if c in name_to_idx:
                idx.append(name_to_idx[c])
            else:
                # Fallback: assume same order for missing names
                idx.append(cols.index(c))
        mean_aligned = mean[idx]
        scale_aligned = scale[idx]
    else:
        # Assume the first N features correspond to the selected columns
        mean_aligned = mean[:len(cols)]
        scale_aligned = scale[:len(cols)]
    X = Xs * scale_aligned + mean_aligned
    out = df.copy()
    out.loc[:, cols] = X
    return out, cols

# Choose columns that were scaled during preprocessing
candidate_cols = ['UTM_x', 'UTM_y', 'SOG', 'v_east', 'v_north']

train_vis, train_cols = inverse_scale_df(train, scaler, candidate_cols, saved_feature_names)
val_vis, val_cols = inverse_scale_df(val, scaler, candidate_cols, saved_feature_names)
print(f"Train unscaled columns: {train_cols}")
print(f"Val unscaled columns: {val_cols}")

In [ ]:
fig = visualize_trajectories(val_vis, color_by='Trajectory')
fig.show()

In [ ]:
# Turn train and val sets into lists of trajectories
train_trajectories = []
    
for traj_id in train['Trajectory'].unique():
    traj_data = train[train['Trajectory'] == traj_id].sort_values('Timestamp')
    features = traj_data[['UTM_x', 'UTM_y', 'SOG', 'v_east', 'v_north']].values
    train_trajectories.append(features)

print(f"Total trajectories: {len(train_trajectories)}")

In [ ]:
val_trajectories = []
    
for traj_id in val['Trajectory'].unique():
    traj_data = val[val['Trajectory'] == traj_id].sort_values('Timestamp')
    features = traj_data[['UTM_x', 'UTM_y', 'SOG', 'v_east', 'v_north']].values
    val_trajectories.append(features)

print(f"Total trajectories: {len(val_trajectories)}")

In [ ]:
class TrajectoryDataset(Dataset):
    def __init__(self, trajectories):
        self.trajectories = trajectories
    
    def __len__(self):
        return len(self.trajectories)
    
    def __getitem__(self, idx):
        traj = torch.FloatTensor(self.trajectories[idx])
        return traj

In [ ]:
def pad_trajectories(batch):
    lengths = torch.tensor([len(traj) for traj in batch])
    padded = pad_sequence(batch, batch_first=True, padding_value=0.0)

    lengths, perm_idx = lengths.sort(descending=True)
    padded = padded[perm_idx]

    return padded, lengths

In [ ]:
train_dataset = TrajectoryDataset(train_trajectories)
val_dataset = TrajectoryDataset(val_trajectories)

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
importlib.reload(RAE)

In [ ]:
# model, train_losses, val_losses = RAE.train_RAE(
#     train_dataset,
#     val_dataset,
#     device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
#     feature_size=5,
#     encoder_layers=3,
#     decoder_layers=2,
#     latent_dim=12,  
#     hidden_size=64,
#     learning_rate=0.001,
#     batch_size=256,
#     max_epochs=50,
#     teacher_forcing_ratio=0.9,
#     dropout=0.1,
#     patience=10
# )

In [ ]:
def hyperparameter_search(
    train_dataset,
    val_dataset,
    device,
    feature_size,
    search_type='grid',
    save_dir='hyperparameter_results'
):
    import os
    import itertools
    import numpy as np
    os.makedirs(save_dir, exist_ok=True)
    
    print("="*70)
    print("HYPERPARAMETER TUNING FOR RAE")
    print("="*70)
    
    # Define search space - FOCUSED on most impactful parameters
    if search_type == 'grid':
        param_grid = {
            # High impact - tune these
            'latent_dim': [8, 12, 16],
            'teacher_forcing_ratio': [0.3, 0.5, 0.7],
            'dropout': [0.1, 0.2],
            'learning_rate': [0.0005, 0.001],
            
            # Fixed - don't tune (saves time)
            'encoder_layers': [3],
            'decoder_layers': [2],
            'hidden_size': [64],
            'batch_size': [256],
            'max_epochs': [25],
            'patience': [8],
            'min_delta': [0.0001]
        }
        # Total: 3 * 3 * 2 * 2 = 36 combinations (~2-3 hours)
        
    elif search_type == 'random':
        param_grid = {
            'latent_dim': [8, 10, 12, 14, 16],
            'teacher_forcing_ratio': [0.3, 0.4, 0.5, 0.6, 0.7],
            'dropout': [0.1, 0.15, 0.2, 0.25, 0.3],
            'learning_rate': [0.0003, 0.0005, 0.001, 0.002],
            'hidden_size': [48, 64, 80],
            
            # Fixed
            'encoder_layers': [3],
            'decoder_layers': [2],
            'batch_size': [256],
            'max_epochs': [25],
            'patience': [8],
            'min_delta': [0.0001]
        }
        
    elif search_type == 'quick':
        # Even faster - just test key variations
        param_grid = {
            'latent_dim': [10, 12],
            'teacher_forcing_ratio': [0.5, 0.7],
            'dropout': [0.1, 0.2],
            'learning_rate': [0.001],
            
            # Fixed
            'encoder_layers': [3],
            'decoder_layers': [2],
            'hidden_size': [64],
            'batch_size': [256],
            'max_epochs': [25],
            'patience': [8],
            'min_delta': [0.0001]
        }
        # Total: 2 * 2 * 2 = 8 combinations (~30-40 min)
    
    # Generate parameter combinations
    if search_type in ['grid', 'quick']:
        keys = param_grid.keys()
        values = param_grid.values()
        combinations = list(itertools.product(*values))
        param_combinations = [dict(zip(keys, combo)) for combo in combinations]
        print(f"Grid Search: {len(param_combinations)} combinations")
        
    elif search_type == 'random':
        n_random = 20
        param_combinations = []
        for _ in range(n_random):
            params = {k: np.random.choice(v) for k, v in param_grid.items()}
            param_combinations.append(params)
        print(f"Random Search: {n_random} combinations")
    
    print(f"Estimated time: {len(param_combinations) * 5} minutes")
    print("="*70)
    
    # Track results
    results = []
    best_score = float('inf')
    best_params = None
    
    start_time = datetime.now()
    
    for idx, params in enumerate(param_combinations):
        print(f"\n{'='*70}")
        print(f"Trial {idx+1}/{len(param_combinations)}")
        print(f"Time elapsed: {(datetime.now() - start_time).total_seconds()/60:.1f} min")
        print(f"{'='*70}")
        print(f"Testing: latent_dim={params['latent_dim']}, "
              f"TF={params['teacher_forcing_ratio']}, "
              f"dropout={params['dropout']}, "
              f"lr={params['learning_rate']}")
        
        try:
            # Train model using your existing function
            model, train_losses, val_losses = RAE.train_RAE(
                train_dataset=train_dataset,
                val_dataset=val_dataset,
                device=device,
                feature_size=feature_size,
                **params  # All params match train_RAE signature now
            )
            
            # Extract metrics from training
            best_val_loss = min(val_losses)
            best_epoch = val_losses.index(best_val_loss) + 1
            final_train_loss = train_losses[best_epoch - 1]
            overfit_ratio = best_val_loss / max(final_train_loss, 1e-6)
            
            # Compute latent space quality
            from torch.utils.data import DataLoader
            
            val_loader = DataLoader(
                val_dataset,
                batch_size=params['batch_size'],
                shuffle=False,
                collate_fn=pad_trajectories
            )
            
            model.eval()
            all_latents = []
            with torch.no_grad():
                for batch_data, lengths in val_loader:
                    batch_data = batch_data.to(device)
                    lengths = lengths.to(device)
                    _, z = model(batch_data, lengths, teacher_forcing_ratio=0.0)
                    all_latents.append(z.cpu())
            
            all_latents = torch.cat(all_latents, dim=0).numpy()
            latent_std = all_latents.std()
            var_per_dim = all_latents.var(axis=0)
            active_dims = (var_per_dim > 0.01).sum()
            
            # Composite score (lower is better)
            score = (
                best_val_loss * 1.0 +                          # Primary metric
                max(0, 0.4 - latent_std) * 0.3 +              # Penalty if std < 0.4
                (params['latent_dim'] - active_dims) * 0.05 +  # Penalty for inactive dims
                max(0, overfit_ratio - 2.0) * 0.2             # Penalty if overfitting > 2x
            )
            
            result = {
                **params,
                'best_val_loss': float(best_val_loss),
                'best_epoch': int(best_epoch),
                'final_train_loss': float(final_train_loss),
                'overfit_ratio': float(overfit_ratio),
                'latent_std': float(latent_std),
                'active_dims': int(active_dims),
                'mean_var_per_dim': float(var_per_dim.mean()),
                'composite_score': float(score),
                'trial': idx + 1
            }
            
            results.append(result)
            
            print(f"\nResults:")
            print(f"  Val Loss: {best_val_loss:.4f}")
            print(f"  Latent Std: {latent_std:.4f}")
            print(f"  Active Dims: {active_dims}/{params['latent_dim']}")
            print(f"  Overfit Ratio: {overfit_ratio:.2f}")
            print(f"  Composite Score: {score:.4f}")
            
            if score < best_score:
                best_score = score
                best_params = params.copy()
                print(f"  ⭐ NEW BEST!")
                
                # Save best model
                torch.save(model.state_dict(), f"{save_dir}/best_model.pth")
            
            # Clean up
            del model
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
        except Exception as e:
            print(f"  ❌ FAILED: {str(e)}")
            import traceback
            traceback.print_exc()
            result = {**params, 'error': str(e), 'trial': idx + 1}
            results.append(result)
    
    # Save results
    if len(results) == 0:
        print("\n❌ No trials completed!")
        return None, None
        
    df = pd.DataFrame(results)
    
    # Only sort by composite_score if there are successful trials
    if 'composite_score' in df.columns:
        df = df.sort_values('composite_score', na_position='last')
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_path = f"{save_dir}/results_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    
    if best_params is None:
        print("\n❌ All trials failed! Check error messages above.")
        return df, None
    
    json_path = f"{save_dir}/best_params_{timestamp}.json"
    with open(json_path, 'w') as f:
        json.dump(best_params, f, indent=2)
    
    # Print summary
    total_time = (datetime.now() - start_time).total_seconds() / 60
    print(f"\n{'='*70}")
    print(f"HYPERPARAMETER SEARCH COMPLETE")
    print(f"{'='*70}")
    print(f"Total time: {total_time:.1f} minutes")
    
    successful = [r for r in results if 'error' not in r]
    print(f"Successful trials: {len(successful)}/{len(results)}")
    
    if len(successful) == 0:
        print("\n❌ No successful trials!")
        return df, None
    
    print(f"\nBest parameters (score={best_score:.4f}):")
    for k, v in best_params.items():
        print(f"  {k}: {v}")
    print(f"\nResults saved to: {csv_path}")
    print(f"Best params saved to: {json_path}")
    print(f"Best model saved to: {save_dir}/best_model.pth")
    
    # Show top 5 (only successful trials)
    print(f"\nTop 5 configurations:")
    successful_df = df[df['composite_score'].notna()]
    top_cols = ['latent_dim', 'teacher_forcing_ratio', 'dropout', 'learning_rate', 
                'best_val_loss', 'latent_std', 'composite_score']
    print(successful_df[top_cols].head())
    
    return df, best_params

In [ ]:
# Quick test (8 combinations, ~30-40 min)
df, best_params = hyperparameter_search(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    feature_size=5,  # your feature size
    search_type='quick'
)

In [19]:
# Full grid search (36 combinations, ~2-3 hours)
df, best_params = hyperparameter_search(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    feature_size=5,
    search_type='grid'
)

HYPERPARAMETER TUNING FOR RAE
Grid Search: 36 combinations
Estimated time: 180 minutes

Trial 1/36
Time elapsed: 0.0 min
Testing: latent_dim=8, TF=0.3, dropout=0.1, lr=0.0005
Training RAE: latent_dim=8, hidden=64, enc_layers=3, dec_layers=2
Model parameters: 217,373

DIAGNOSTICS - Epoch 1
Latent space statistics:
  Mean (avg across dims): 0.0113
  Std (overall): 0.6688 (should be > 0.3)
  Active dimensions: 8/8 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=-0.605, std=0.441, var=0.1942 ✓
  Dim 1: mean=+0.016, std=0.647, var=0.4191 ✓
  Dim 2: mean=+0.129, std=0.638, var=0.4066 ✓
  Dim 3: mean=-0.026, std=0.745, var=0.5554 ✓
  Dim 4: mean=+0.028, std=0.720, var=0.5178 ✓
  Dim 5: mean=+0.294, std=0.546, var=0.2986 ✓
  Dim 6: mean=+0.167, std=0.606, var=0.3678 ✓
  Dim 7: mean=+0.089, std=0.560, var=0.3137 ✓

Epoch 1/25 [TF=0.3] - Train: 4.9065, Val: 5.1384 ✓
Epoch 2/25 [TF=0.3] - Train: 4.6232, Val: 5.1026 ✓
Epoch 3/25 [TF=0.3] - Train: 4.1437, Val: 4.9972 ✓
Epoch 4/25 [TF=0.3] - T

Traceback (most recent call last):
  File "/var/folders/rk/ynqvn_8d77d8956x_nf3jdsc0000gp/T/ipykernel_16023/3316202406.py", line 128, in hyperparameter_search
    from models.RAE import pad_trajectories
ModuleNotFoundError: No module named 'models'



DIAGNOSTICS - Epoch 1
Latent space statistics:
  Mean (avg across dims): 0.0083
  Std (overall): 0.6582 (should be > 0.3)
  Active dimensions: 8/8 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=-0.342, std=0.500, var=0.2501 ✓
  Dim 1: mean=-0.044, std=0.786, var=0.6172 ✓
  Dim 2: mean=+0.005, std=0.649, var=0.4211 ✓
  Dim 3: mean=+0.345, std=0.511, var=0.2607 ✓
  Dim 4: mean=-0.052, std=0.698, var=0.4876 ✓
  Dim 5: mean=+0.005, std=0.520, var=0.2706 ✓
  Dim 6: mean=+0.095, std=0.721, var=0.5198 ✓
  Dim 7: mean=+0.054, std=0.622, var=0.3863 ✓

Epoch 1/25 [TF=0.3] - Train: 4.8531, Val: 5.1227 ✓
Epoch 2/25 [TF=0.3] - Train: 4.1824, Val: 5.0693 ✓
Epoch 3/25 [TF=0.3] - Train: 2.8529, Val: 5.3740 (patience: 1/8)
Epoch 4/25 [TF=0.3] - Train: 1.7043, Val: 4.4878 ✓

DIAGNOSTICS - Epoch 5
Latent space statistics:
  Mean (avg across dims): 0.0061
  Std (overall): 0.6775 (should be > 0.3)
  Active dimensions: 8/8 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=+0.048, std=0.653, var=

Traceback (most recent call last):
  File "/var/folders/rk/ynqvn_8d77d8956x_nf3jdsc0000gp/T/ipykernel_16023/3316202406.py", line 128, in hyperparameter_search
    from models.RAE import pad_trajectories
ModuleNotFoundError: No module named 'models'



DIAGNOSTICS - Epoch 1
Latent space statistics:
  Mean (avg across dims): 0.0177
  Std (overall): 0.6647 (should be > 0.3)
  Active dimensions: 8/8 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=+0.023, std=0.637, var=0.4054 ✓
  Dim 1: mean=-0.176, std=0.811, var=0.6583 ✓
  Dim 2: mean=-0.080, std=0.715, var=0.5113 ✓
  Dim 3: mean=+0.003, std=0.591, var=0.3490 ✓
  Dim 4: mean=+0.181, std=0.649, var=0.4216 ✓
  Dim 5: mean=+0.193, std=0.485, var=0.2356 ✓
  Dim 6: mean=-0.038, std=0.568, var=0.3224 ✓
  Dim 7: mean=+0.034, std=0.723, var=0.5228 ✓

Epoch 1/25 [TF=0.3] - Train: 4.9330, Val: 5.1416 ✓
Epoch 2/25 [TF=0.3] - Train: 4.6258, Val: 5.1108 ✓
Epoch 3/25 [TF=0.3] - Train: 4.2260, Val: 5.0706 ✓
Epoch 4/25 [TF=0.3] - Train: 3.6094, Val: 4.6434 ✓

DIAGNOSTICS - Epoch 5
Latent space statistics:
  Mean (avg across dims): 0.0113
  Std (overall): 0.6834 (should be > 0.3)
  Active dimensions: 8/8 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=+0.078, std=0.769, var=0.5916 ✓
  Dim

...

Traceback (most recent call last):
  File "/var/folders/rk/ynqvn_8d77d8956x_nf3jdsc0000gp/T/ipykernel_16023/3316202406.py", line 128, in hyperparameter_search
    from models.RAE import pad_trajectories
ModuleNotFoundError: No module named 'models'



DIAGNOSTICS - Epoch 1
Latent space statistics:
  Mean (avg across dims): -0.0171
  Std (overall): 0.6439 (should be > 0.3)
  Active dimensions: 12/12 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=-0.040, std=0.505, var=0.2548 ✓
  Dim 1: mean=+0.118, std=0.608, var=0.3701 ✓
  Dim 2: mean=-0.301, std=0.484, var=0.2339 ✓
  Dim 3: mean=+0.112, std=0.678, var=0.4603 ✓
  Dim 4: mean=-0.170, std=0.500, var=0.2501 ✓
  Dim 5: mean=-0.142, std=0.610, var=0.3723 ✓
  Dim 6: mean=+0.089, std=0.711, var=0.5051 ✓
  Dim 7: mean=-0.070, std=0.672, var=0.4518 ✓
  Dim 8: mean=+0.276, std=0.437, var=0.1907 ✓
  Dim 9: mean=+0.061, std=0.770, var=0.5924 ✓
  Dim 10: mean=+0.064, std=0.769, var=0.5911 ✓
  Dim 11: mean=-0.201, std=0.633, var=0.4007 ✓

Epoch 1/25 [TF=0.7] - Train: 4.6247, Val: 5.1095 ✓
Epoch 2/25 [TF=0.7] - Train: 3.2127, Val: 4.7946 ✓
Epoch 3/25 [TF=0.7] - Train: 1.8319, Val: 4.6171 ✓
Epoch 4/25 [TF=0.7] - Train: 1.0858, Val: 4.4900 ✓

DIAGNOSTICS - Epoch 5
Latent space statistics:
  

Traceback (most recent call last):
  File "/var/folders/rk/ynqvn_8d77d8956x_nf3jdsc0000gp/T/ipykernel_16023/3316202406.py", line 128, in hyperparameter_search
    from models.RAE import pad_trajectories
ModuleNotFoundError: No module named 'models'



DIAGNOSTICS - Epoch 1
Latent space statistics:
  Mean (avg across dims): 0.0087
  Std (overall): 0.6560 (should be > 0.3)
  Active dimensions: 12/12 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=-0.012, std=0.698, var=0.4869 ✓
  Dim 1: mean=+0.052, std=0.514, var=0.2638 ✓
  Dim 2: mean=-0.093, std=0.662, var=0.4382 ✓
  Dim 3: mean=-0.074, std=0.635, var=0.4033 ✓
  Dim 4: mean=+0.085, std=0.591, var=0.3491 ✓
  Dim 5: mean=+0.184, std=0.577, var=0.3331 ✓
  Dim 6: mean=+0.036, std=0.733, var=0.5370 ✓
  Dim 7: mean=+0.128, std=0.672, var=0.4511 ✓
  Dim 8: mean=-0.098, std=0.680, var=0.4617 ✓
  Dim 9: mean=-0.103, std=0.683, var=0.4664 ✓
  Dim 10: mean=+0.258, std=0.623, var=0.3885 ✓
  Dim 11: mean=-0.257, std=0.598, var=0.3576 ✓

Epoch 1/25 [TF=0.7] - Train: 4.7064, Val: 5.1433 ✓
Epoch 2/25 [TF=0.7] - Train: 4.1270, Val: 5.1033 ✓
Epoch 3/25 [TF=0.7] - Train: 3.4335, Val: 5.0183 ✓
Epoch 4/25 [TF=0.7] - Train: 2.5929, Val: 4.2559 ✓

DIAGNOSTICS - Epoch 5
Latent space statistics:
  M

KeyboardInterrupt: 

In [23]:
df

,latent_dim,teacher_forcing_ratio,dropout,learning_rate,encoder_layers,decoder_layers,hidden_size,batch_size,max_epochs,patience,min_delta,error,trial
0,10,0.5,0.1,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,1
1,10,0.5,0.2,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,2
2,10,0.7,0.1,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,3
3,10,0.7,0.2,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,4
4,12,0.5,0.1,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,5
5,12,0.5,0.2,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,6
6,12,0.7,0.1,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,7
7,12,0.7,0.2,0.001,3,2,64,256,25,8,0.0001,name 'RAE_weird' is not defined,8


In [ ]:
model, train_losses, val_losses = RAE.train_RAE(
    train_dataset,
    val_dataset,
    device=device,
    feature_size=5,
    encoder_layers=3,
    decoder_layers=2,
    latent_dim=12,  
    hidden_size=64,
    learning_rate=0.001,
    batch_size=256,
    max_epochs=50,
    teacher_forcing_ratio=0.3,
    dropout=0.2,
    patience=10
)

Training RAE: latent_dim=12, hidden=64, enc_layers=3, dec_layers=2
Model parameters: 218,409

DIAGNOSTICS - Epoch 1
Latent space statistics:
  Mean (avg across dims): 0.0122
  Std (overall): 0.6681 (should be > 0.3)
  Active dimensions: 12/12 (var > 0.01)

Per-dimension statistics:
  Dim 0: mean=-0.029, std=0.762, var=0.5806 ✓
  Dim 1: mean=-0.119, std=0.632, var=0.3990 ✓
  Dim 2: mean=+0.092, std=0.699, var=0.4892 ✓
  Dim 3: mean=+0.024, std=0.606, var=0.3676 ✓
  Dim 4: mean=-0.070, std=0.699, var=0.4887 ✓
  Dim 5: mean=-0.074, std=0.743, var=0.5526 ✓
  Dim 6: mean=+0.080, std=0.737, var=0.5433 ✓
  Dim 7: mean=-0.101, std=0.591, var=0.3496 ✓
  Dim 8: mean=-0.067, std=0.564, var=0.3179 ✓
  Dim 9: mean=+0.018, std=0.630, var=0.3968 ✓
  Dim 10: mean=-0.077, std=0.662, var=0.4386 ✓
  Dim 11: mean=+0.469, std=0.391, var=0.1526 ✓

Epoch 1/50 [TF=0.3] - Train: 4.7938, Val: 5.1039 ✓
Epoch 2/50 [TF=0.3] - Train: 4.0737, Val: 4.8045 ✓
Epoch 3/50 [TF=0.3] - Train: 3.0658, Val: 5.5657 (patience: 